In [5]:
import altair as alt
import pandas as pd
import unicodedata

alt.data_transformers.disable_max_rows()

source = pd.read_csv("/content/base-games.csv", sep=",", encoding='latin-1')

source.head()

,Compañia,Videojuego,Año,Consola/PC,N° Trabajadores,Tamaño del juego (Aprox.),Original/Remaster/Remake/Expansión,Estética,Puntaje Medio Críticas (1-100)
0,Naughty Dog,Crash Bandicoot,1996,PS1,84,428 MB,Original,Cartoon,80
1,Naughty Dog,Crash Bandicoot 2: Cortex Strikes Back,1997,PS1,122,71.19 MB,Original,Cartoon,89
2,Naughty Dog,Crash Bandicoot 3: Warped,1998,PS1,204,76.77 MB,Original,Cartoon,91
3,Naughty Dog,Crash Team Racing,1999,PS1,199,267.8 MB,Original,Cartoon,88
4,Naughty Dog,Jak and Daxter: The Precursor´s Legacy,2001,PS2,277,977 MB,Original,Cartoon,90


In [6]:
import altair as alt
import pandas as pd
import unicodedata


filtro = "Naughty Dog"
data_filtrada = source[
    (source['Compañia'] == filtro) &
    (source['Original/Remaster/Remake/Expansión'].str.lower().str.strip() == 'original')
].copy()

def normalizar_texto(texto):
    if isinstance(texto, str):
        texto = texto.strip()
        texto = ''.join(
            c for c in unicodedata.normalize('NFD', texto)
            if unicodedata.category(c) != 'Mn'
        )
    return texto

data_filtrada['Videojuego'] = data_filtrada['Videojuego'].apply(normalizar_texto)

data_filtrada['Tamaño del juego (Aprox.)'] = (
    data_filtrada['Tamaño del juego (Aprox.)']
    .astype(str)
    .str.strip()
    .replace(['nan', ''], "No disponible")
    .fillna("No disponible")
)

data_filtrada = data_filtrada.sort_values('Año', ascending=True)

data_filtrada['Videojuego'] = pd.Categorical(
    data_filtrada['Videojuego'],
    categories=data_filtrada['Videojuego'].tolist(),
    ordered=True
)

data_filtrada['Tamaño del juego (Aprox.)'].head()



,Tamaño del juego (Aprox.)
0,428 MB
1,71.19 MB
2,76.77 MB
3,267.8 MB
4,977 MB


In [12]:
import altair as alt
import pandas as pd
import unicodedata

chart = alt.Chart(data_filtrada).mark_circle(size=100).encode(
    x=alt.X("Año:Q",
            scale=alt.Scale(domain=[1995, 2030]),
            axis=alt.Axis(tickCount=35, title="Año de lanzamiento")),
    y=alt.Y("Videojuego:N",
            sort=data_filtrada['Videojuego'].tolist(),
            title="Videojuego"),
    tooltip=[
        "Año",
        "Consola/PC",
        "N° Trabajadores",
        "Tamaño del juego (Aprox\\.)"
    ]
).properties(
    width=700,
    height=500,
    title=f"Juegos originales de {filtro} (1995–2030)"
).interactive()

chart

alt.Chart(...)

In [14]:
chart.save("juegos_naughty_dog.html")